In [3]:
import numpy as np
import pandas as pd

In [4]:
def get_std():
    df_init = pd.read_csv('../aig/aiginit.csv',header=None)
    df_init = df_init.rename(columns={0:'aig',1:'num',2:'lev'})
    std = {}
    for line in df_init.itertuples():
        std[line[1]]=(line[2],line[3])
    df_std = pd.read_excel('../aig/aig.xlsx')
    for line in df_std.itertuples():
        if line[2] not in std:
            std[line[2]]=(0,0)
        std[line[2]] = (*std[line[2]],line[4],line[3])
    return std

def read_log(file,learn_start=5e4,cn=0.5,cl=1):
    std=get_std()
    df = pd.read_csv(file,header=None)
    df = df.rename(columns={0:'aig',1:'type',2:'num',3:'lev',4:'returns',5:'length'})
    done = df[df['type']=='done']
    dl = done['length']
    count = 0
    i=0
    while count<learn_start:
        count += 20
        i+=1
    df = df.iloc[dl.index[i]:]
    df = df[df['type']=='max']
    df = df.loc[:,['aig','num','lev']]
    df.loc[:,'num_init'] = 0
    df.loc[:,'lev_init'] = 0
    df.loc[:,'num_std'] = 0
    df.loc[:,'lev_std'] = 0
    df.loc[:,'value'] = 0.0
    df.loc[:,'value_std'] = 0.0
    df.loc[:,'diff'] = 0.0
    df.loc[:,'score'] = 0.0
    df.loc[:,'score_std'] = 0.0
    for i in range(len(df)):
        line = df.iloc[i]
        aig = line['aig']
        if aig in std and std[aig][0]!=0 and std[aig][1] != 0:
            line['num_init']=std[aig][0]
            line['lev_init']=std[aig][1]
            line['num_std']=std[aig][2]
            line['lev_std']=std[aig][3]
            line['value']=1 - (line['num']/line['num_init'])**cn*(line['lev']/line['lev_init'])**cl
            line['value_std']=1 - (line['num_std']/line['num_init'])**cn*(line['lev_std']/line['lev_init'])**cl
            line['diff'] = line['value']-line['value_std']
            line['score'] = (0.8 if line['num']<=line['num_std'] else 0.4) + (1.2 if line['lev']<=line['lev_std'] else 0.6)
            line['score_std'] = (0.8 if line['num']>=line['num_std'] else 0.4) + (1.2 if line['lev']>=line['lev_std'] else 0.6)
            df.iloc[i] = line
    df.sort_values(by='value',inplace=True,ascending=False,kind='stable')
    df.sort_values(by='aig',inplace=True,ascending=True,kind='stable')
    df = df.drop_duplicates(subset='aig')
    return df

def reads(outfile:str,*paths:str):
    aigs = []
    for path in paths:
        aigs.append(read_log(path))
    with pd.ExcelWriter(outfile) as writer:
        for i,df in enumerate(aigs):
            df.to_excel(writer,sheet_name=str(i))
# get_std()
reads('aig2.1.xlsx','/home/jiaxingwang/code/aig20/log/1218_1718.csv')

/tmp/ipykernel_200677/3082105092.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  line['num_init']=std[aig][0]
/tmp/ipykernel_200677/3082105092.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  line['lev_init']=std[aig][1]
/tmp/ipykernel_200677/3082105092.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  line['num_std']=std[aig][2]
/tmp/ipykernel_200677/3082105092.py:44: SettingWithCopyWarning: 
A value is trying to be set on